In [1]:
# deepseek模型的api_key
dp_api_key = "sk-5c17979d4ce24427880fbd1dd107ecd5"
data_batch = "sample"

In [2]:
import asyncio
from openai import AsyncOpenAI
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas
import math
from tqdm.asyncio import tqdm_asyncio
from tqdm.notebook import tqdm

In [3]:
# 全局初始化客户端与菜单信息
aclient = AsyncOpenAI(
    api_key=dp_api_key,
    base_url="https://api.deepseek.com"
)

In [4]:
# 大模型调用函数
def get_intro(comment):
    client = OpenAI(api_key=dp_api_key, base_url="https://api.deepseek.com")
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "你是一名精明的房产从业者，熟知评判一处房屋好坏的各种条件"},
            {"role": "user", "content": f"假设将房屋硬件条件良好、配套设施齐全、周边条件便利定义为“好”，将配套设施不全、条件恶劣定义为“不好”，【{comment}】这句评价中的房屋更贴近“很好、比较好、适中、比较不好、很不好”这五种选项中的哪一种？请告诉我五个选项中最合适的一个。我需要依据你的回答进行分类，因此不要修改选项，不要解释理由。"}
        ],
        stream=False
    )
    return response.choices[0].message.content

In [5]:
# com = "电费按表计量，治安堪忧，拎包入住"
# print(get_intro(com))

In [6]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1))
async def async_get_intro(comment, semaphore):
    async with semaphore:  # 控制并发量
        response = await aclient.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "你是一名精明的房产从业者，熟知评判一处房屋好坏的各种条件"},
                {"role": "user", "content": f"假设将房屋硬件条件良好、配套设施齐全、周边条件便利定义为“好”，将配套设施不全、条件恶劣定义为“不好”，【{comment}】这句评价中的房屋更贴近“很好、比较好、适中、比较不好、很不好”这五种选项中的哪一种？请告诉我五个选项中最合适的一个。我需要依据你的回答进行分类，因此不要修改选项，不要解释理由。"}
            ],
            stream=False
        )
        return response.choices[0].message.content

In [7]:
import asyncio
from tqdm.asyncio import tqdm_asyncio

async def batch_get_intros(items, max_concurrency=50):
    semaphore = asyncio.Semaphore(max_concurrency)
    
    async def bounded_get_intro(item):
        async with semaphore:
            # 添加缺失的 semaphore 参数
            return await async_get_intro(item, semaphore)
    
    tasks = [bounded_get_intro(item) for item in items]
    
    results = await tqdm_asyncio.gather(
        *tasks,
        desc=f"Processing: (max_concurrency={max_concurrency})",
        total=len(items),
        ascii=True,
        mininterval=0.1
    )
    return results

In [8]:
data_batch = "train"
data_target = "price"
path = f"结构化数据_{data_batch}_{data_target}.csv"  # 存有菜品名的csv文件地址
df = pandas.read_csv(path)

In [9]:
batch_size = 1000  # 单次处理量
comment_column = "客户反馈"  # 存有菜品名的列名
id_column = "Price"
n_dim = df.shape[0]

In [10]:
# 分批次获取结果
for epoch in range(math.ceil(n_dim / batch_size)):
    start_idx = epoch * batch_size
    end_idx = min(start_idx + batch_size, n_dim)
    
    # 获取当前批次菜品名
    batch_comments = df.iloc[start_idx:end_idx][comment_column].tolist()
    batch_ids = df.iloc[start_idx:end_idx][id_column].tolist()
#     for item in batch_items:
#         item = str(item)
    
    # 异步获取菜品标签
    feel_results = await batch_get_intros(batch_comments) # 确保返回顺序与 batch_items 一致
    
    # 构建DataFrame并进行二次检查
    batch_intro_df = pandas.DataFrame({
        "ID": batch_ids,
        "客户反馈": batch_comments,
        "客户反馈评级": feel_results
    })
    
    # 保存文件
    output_path = f"batch_result//客户反馈评级//{data_batch}_{data_target}//epoch{epoch}.csv"
    batch_intro_df.to_csv(output_path, index=False)
    
    # 更新进度
    print(f"Processed: {epoch+1}/{math.ceil(n_dim / batch_size)} batches")

Processing: (max_concurrency=50): 100%|##########| 1000/1000 [00:41<00:00, 23.88it/s]


Processed: 1/104 batches


Processing: (max_concurrency=50): 100%|##########| 1000/1000 [00:41<00:00, 24.31it/s]


Processed: 2/104 batches


Processing: (max_concurrency=50): 100%|##########| 1000/1000 [00:41<00:00, 23.84it/s]


...

Processed: 102/104 batches


Processing: (max_concurrency=50): 100%|##########| 1000/1000 [00:43<00:00, 23.18it/s]


Processed: 103/104 batches


Processing: (max_concurrency=50): 100%|##########| 871/871 [00:37<00:00, 23.02it/s]


Processed: 104/104 batches
